Extracting Statistical features like - 
1. TF-IDF
2. Doc2Vec 
3. Word2Vec 
4. Bert
5. Handcrafted(ref to extract_handcrafted_feature.ipynb)

In [ ]:
from google.colab import drive 
drive.mount('/gdrive')

Mounted at /gdrive


# TF-IDF (Ref. for Text_hero https://stackoverflow.com/questions/37593293/how-to-get-tfidf-with-pandas-dataframe)

In [ ]:
# Load the lemmatized features - 
import pandas as pd 
import json
## json structure of - cleanSynopsis_ageRange.json = 
##{'dataset': [{  'desc': ["text1", "text2"]
##              'lower_age': int
##              'upper_age': int
##}]
##    
##}


# Read Json 

# Open the cleanSynopsis_ageRange.json file
with open("/gdrive/My Drive/MSC_Thesis_Project/Data/cleanSynopsis_ageRange.json") as file:
    # Load its content and make a new dictionary
    lemm_data = json.load(file)

#lemm_data['dataset'][12]['desc']

# on lemmatized

from sklearn.feature_extraction.text import TfidfVectorizer 

documents_df=pd.DataFrame(lemm_data['dataset'],columns=['desc','desc_sentences'])

#concatenating all rows of lemmatized data into a series of words - 
allSentences= []
sentence = ""
for i in documents_df.index:
         sentence = ""
         for a in  documents_df['desc'][i]:
             sentence =  sentence + a +" " 
         documents_df['desc_sentences'][i] = sentence
         #allSentences.append(sentence) 

#documents_df
!pip install texthero

import texthero as hero
documents_df['tfidf'] = hero.tfidf(documents_df['desc_sentences'])

documents_df.head()
#tfidfvectoriser=TfidfVectorizer()
#tfidfvectoriser.fit(allSentences)
#tfidf_vectors=tfidfvectoriser.transform(allSentences)
#matrix = tfidfvectoriser.fit_transform(allSentences)


#matrix
#print("Feature Names ",len(tfidfvectoriser.get_feature_names_out()))
#tfidf_vectors 

FileNotFoundError: ignored

Conclusion from TF-IDF - 
1. the outcome of tfidf column in the above code gave too many 0.0s 
2. This is because none of the words from one description match with other. 
3. Hence, we don't want to use tfidf :

Word2vec - 

In [ ]:
#Loading non-lemmatized data from 'cleanSynopsis_ageRange_nonlemm.json'
# Opening the file 
with open("/gdrive/My Drive/MSC_Thesis_Project/Data/cleanSynopsis_ageRange_nonlemm.json") as file:
    # Load its content and make a new dictionary
    data = json.load(file)

#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM'

#installing word2vec dictionary

!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
     # https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2022-04-10 15:20:31--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.80.211
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.80.211|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-04-10 15:20:31 ERROR 404: Not Found.



In [ ]:
##library for text- processing 
!pip install gensim

In [ ]:
# loading pre-trained embeddings, each word is represented as a 300 dimensional vector
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/gdrive/My Drive/MSC_Thesis_Project/Data/GoogleNews-vectors-negative300.bin.gz', binary=True)


In [ ]:
def filter_docs(corpus, texts, condition_on_doc):
    # Filter corpus, texts and labels given the function condition_on_doc which takes
    #a doc.
    #The document doc is kept if condition_on_doc(doc) is true.
    
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]

    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)


In [ ]:
import numpy as np 

def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    return np.mean(word2vec_model[doc], axis=0)
   # return np.sum(word2vec_model[doc], axis=0)


In [ ]:
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)




In [ ]:
docs_df=pd.DataFrame(data['dataset'],columns=['desc'])

x =[]
for i in docs_df.index: #look up each doc in model
    x.append(document_vector(model, docs_df['desc'][i]))
    
X= np.array(x)

np.shape(X)


(1581, 300)

In [ ]:
# Saving the word2vec features - 
np.save('/gdrive/My Drive/MSC_Thesis_Project/Data/word2vec_mean_features', X)  #will be saved as .npy file